In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
rawdata1 = pd.read_excel("../tourdataset/tourdata2017_1.xlsx")
rawdata2 = pd.read_excel("../tourdataset/tourdata2017_2.xlsx")
rawdata3 = pd.read_excel("../tourdataset/tourdata2017_3.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '../tourdataset/tourdata2017_1.xlsx'

In [ ]:
# 컬럼명 통일 (공백제거, 소문자 맞춤)

rawdata1.rename(columns=lambda x: x.strip().lower(), inplace=True)
rawdata2.rename(columns=lambda x: x.strip().lower(), inplace=True)
rawdata3.rename(columns=lambda x: x.strip().lower(), inplace=True)
# rawdata3.columns = map(str.lower, rawdata3.columns)

### 분석 데이터 생성

1-1. 데이터 정제

In [ ]:
# 1. m_id 빈 값 지우기

rawdata2 = rawdata2.dropna(subset=["m_id"])
rawdata3 = rawdata3.dropna(subset=["m_id"])

In [ ]:
# 국내여행자만 보기

rawdata2 = rawdata2[rawdata2["type1"] == 1]
rawdata3 = rawdata3[rawdata3["type1"] == 1]

In [ ]:
# 필요한 컬럼만 추출

rawdata1 = rawdata1.drop(['sr_type','ara_size', 'school1','school2','marry','inc2_1','wt'], axis = 1)

In [ ]:
rawdata2 = rawdata2[["hid","pid","type1","month","q1","q2_c_1","q2_c_2","q3","q4_a",
                       "q5","q5_1","q5_2_1","q5_2_2","q5_2_3","q5_2_4","q5_2_5","q5_2_6",
                      "q7","q7_1","q7_3_1","q7_3_2","q7_3_3","q7_3_4","q7_3_5","q7_3_6","q7_3_7","q7_3_8","q7_3_9","q7_3_10",
                      "q8","q8_1","q8_2","q8_3","q8_4","q8_5","q8_6","q8_7",
                      "q10_1","q10_2","q10_3","q10_4","q10_5","q10_6","q10_7","q10_8","q10_9","q10_10","q10_11","q10_12"]]

In [ ]:
rawdata3 = rawdata3.drop(['m_id','type2','q6_1_7','q6_1_8','q6_5_1a','q6_5_2a','q6_5_3a','q6_5_4a','q6_5_5a','q6_5_6a','q6_5_7a','q6_5_8a','q6_5_9a','q6_5_10a','q6_5_11a','q6_5_12a','q6_5_13a', 'wt'], axis = 1)

1-2. 1차 join = 단위여행 + 방문지

In [ ]:
# 1차 join = 같은 응답자의 단위여행 데이터와, 방문지 데이터를 합친다.

joinKey = ["hid","pid","month"]

In [ ]:
firstmerge = pd.merge(rawdata2, rawdata3, left_on=joinKey, right_on=joinKey, how="left")

2-1. 응답자 20대 추출

In [ ]:
rawdata1["age_group"] = np.where(rawdata1["age"]>0, (((rawdata1["age"]/10).astype(int))*10) , 0)

In [ ]:
rawdata1 = rawdata1[rawdata1.age_group==20]

2-2. 2차 join = 응답자 + 1차 join

In [ ]:
# 2차 join 데이터 = 같은 응답자의 응답자 정보와 1차 join 데이터를 합친다.

joinKey2 = ["hid","pid"]

In [ ]:
secondmerge = pd.merge(rawdata1, firstmerge, left_on=joinKey2, right_on=joinKey2, how="left")

In [ ]:
secondmerge = secondmerge.dropna(subset=["q7"])

In [ ]:
secondmerge

### 만족도 기준 새로 생성하기

In [ ]:
satisfaction = secondmerge.dropna(subset=["q10_1"]).reset_index(drop = True)

In [ ]:
# 항목별 만족도에서 9.경험해보지 못함 -> 0으로 바꿈

satisfaction.loc[:,"q10_1":"q10_12"] = np.where(satisfaction.loc[:,"q10_1":"q10_12"] == 9,0,satisfaction.loc[:,"q10_1":"q10_12"])

In [ ]:
tempList = []
for i in range(0,len(satisfaction)):
    zeroCount = 0
    for j in satisfaction.loc[i,"q10_1":"q10_12"]:
    
        if j==0:
            zeroCount += 1
    tempList.append(
     (round((math.pow((sum(satisfaction.loc[i,"q10_1":"q10_12"]) / (12-zeroCount)), 2)) * 10) / 10) * 1.2
     + (math.pow((satisfaction.loc[i,"q6_7"]), 2) * 1.6)
     + (math.pow((satisfaction.loc[i,"q6_8"]), 2) * 0.8)
     + (math.pow((satisfaction.loc[i,"q6_9"]), 2) * 0.4))

In [ ]:
colName = "q10_grade"
tempDF = pd.DataFrame(tempList, columns=[colName])
result01 = pd.concat([satisfaction, tempDF], axis=1)

In [ ]:
satisfaction2 = secondmerge.dropna(subset=["q10_1"]).reset_index(drop = True)

In [ ]:
tempList = []
for i in range(0,len(satisfaction2)):
    tempList.append(round((sum(satisfaction2.loc[i,"q6_7":"q6_9"])/3)*2))

In [ ]:
colName = "q10_grade2"
tempDF = pd.DataFrame(tempList, columns=[colName])
result02 = pd.concat([satisfaction2, tempDF], axis=1)

In [ ]:
result01

In [ ]:
result01.to_csv("../tourdataset/result01.csv")

In [ ]:
'q10_grade'
"hid", "pid", "sex" ,"age" ,"sido" ,"occ1" ,"inc1_1", 'month'
성별, 나이, 거주시도, 직업, 연평군가구소득
"q1","q5","q7","q6_1","q6_4"
당일/숙박, 일행존재여부, 총지출비용, 여행한광역시, 이동수단

In [ ]:
corrGroupKey = ["hid","pid",'month','age',"occ1" ]

In [ ]:
tmp = result01.groupby(corrGroupKey).sum().reset_index(drop=False)

In [ ]:
columnList = ['q10_grade','hid', 'pid', 'month', 'age', 'sex', 'sido', 'occ1', 'occ2', 'fac', 'inc1_1', 'age_group', 'type1_x', 'q1', 'q2_c_1', 'q2_c_2', 'q3', 'q4_a', 'q5', 'q5_1', 'q5_2_1', 'q5_2_2', 'q5_2_3', 'q5_2_4', 'q5_2_5', 'q5_2_6', 'q7', 'q7_1', 'q7_3_1', 'q7_3_2', 'q7_3_3', 'q7_3_4', 'q7_3_5', 'q7_3_6', 'q7_3_7', 'q7_3_8', 'q7_3_9', 'q7_3_10', 'q8', 'q8_1', 'q8_2', 'q8_3', 'q8_4', 'q8_5', 'q8_6', 'q8_7', 'q10_1', 'q10_2', 'q10_3', 'q10_4', 'q10_5', 'q10_6', 'q10_7', 'q10_8', 'q10_9', 'q10_10', 'q10_11', 'q10_12', 'type1_y', 'q6_1', 'q6_1_1', 'q6_1_2', 'q6_1_3', 'q6_1_4', 'q6_1_5', 'q6_1_6', 'q6_2', 'q6_3_a', 'q6_3_b', 'q6_3_c', 'q6_4', 'q6_5_1', 'q6_5_2', 'q6_5_3', 'q6_5_4', 'q6_5_5', 'q6_5_6', 'q6_5_7', 'q6_5_8', 'q6_5_9', 'q6_5_10', 'q6_5_11', 'q6_5_12', 'q6_5_13', 'q6_6_1', 'q6_6_2', 'q6_6_3', 'q6_6_4', 'q6_6_5', 'q6_6_6', 'q6_6_7', 'q6_6_8', 'q6_6_9', 'q6_6_10', 'q6_6_11', 'q6_6_12', 'q6_6_13', 'q6_6_14', 'q6_6_15', 'q6_6_16', 'q6_6_17', 'q6_6_18', 'q6_6_19', 'q6_6_20', 'q6_6_21', 'q6_7', 'q6_8', 'q6_9', 'q7_2',]

In [ ]:
tmp = tmp[columnList]

In [ ]:
corrDF999 = tmp.corr()

In [ ]:
corrDF999

In [ ]:
corrDF999 = tmp.corr()
corrDF999.to_csv("../tourdataset/corr001.csv")

In [ ]:
tempList = []
for i in range(0,len(result01)):
    jobCode = 1231231247
    j = result01["occ1"][i]
    if ((j == 14) | (j == 15)):
        #대학생
        jobCode = 0
    elif j < 10:
        #직장인
        jobCode = 1
    elif j == 16:
        #무직
        jobCode = 2
    elif j == 10:
        #군인
        jobCode = 3
    else:
        jobCode = 4
    tempList.append(jobCode)

In [ ]:
colName = "job_group"
tempDF = pd.DataFrame(wowList, columns=[colName])
result02 = pd.concat([result01, tempDF], axis=1)

In [ ]:
result02.to_csv("../tourdataset/result02.csv")

In [ ]:
result02["prop_id"] = result02["hid"]+result02["pid"]+result02["month"]

In [ ]:
result02

### K-Means Clustering

In [ ]:
import pandas as pd
# K-Means 라이브러리
from sklearn.cluster import KMeans
# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Only needed if you want to display your plots inline if using Notebook
# change inline to auto if you have Spyder installed
%matplotlib inline

In [ ]:
def elbow(X):
    sse = []
    for i in range(1,11):
        km = KMeans(n_clusters=i, init="k-means++", random_state=0)
        km.fit(X)
        sse.append(km.inertia_)
    plt.plot(range(1,11),sse,marker='o')
    plt.xlabel('cluster number')
    plt.ylabel('SSE')
    plt.show()

In [ ]:
### 기본정보 가지고 있기
base = result02[["prop_id","q10_grade","job_group"]]
### 특성정보 분리
features = result02[["q1","q3" ,"q4_a","q5_1","q7", "q8"]]
# "q1","q3" ,"q4_a","q5_1","q7", "q8"
# q1 당일/숙박여행여부
# q3 여행목적
# q4_a 정보획득지
# q5_1 동행일행수
# q7 동행일행수
# q8 사전예약여부

In [ ]:
# 클러스터 갯수 정하기
elbow(features)

In [ ]:
model = KMeans(n_clusters=2, algorithm='auto')
model.fit(features)
### ATL (Above The Line) 가시적, 
### BTL (Below The Line) 비가시적, 신문광고 등

In [ ]:
predict = pd.DataFrame(model.predict(features))
predict.columns=["cluster"]

In [ ]:
result03 = pd.concat([base, features, predict], axis = 1)
result03

In [ ]:
job0 = result03[result03["job_group"]==0]   #대학생
job1 = result03[result03["job_group"]==1]   #직장인

job2 = result03[result03["job_group"]==2]   #무직
job3 = result03[result03["job_group"]==3]   #군인

In [ ]:
xAxis = job1["q7"]
yAxis = job1["q10_grade"]
cluster = job1["cluster"]

In [ ]:
plt.scatter(xAxis, yAxis, c=cluster, alpha=0.5)

centers = pd.DataFrame(model.cluster_centers_,columns=['q1','q3'])
center_x = centers['q1']
center_y = centers['q3']
plt.scatter(center_x,center_y,s=50,marker='D',c='r')
plt.show()

In [ ]:
# result01 = rawdata 세 개 합친 버전
# result02 = result01 + job group 버전
# result03 = clustering 버전
# result04 = result01 + job group + clustering + q7 grade 버전

In [ ]:
result04 = pd.concat([result02, predict], axis = 1)
result04.columns

In [ ]:
# .reset_index(drop=False)

In [ ]:
max(result04["q7"])

In [ ]:
tmp = result04.groupby(["q7"]).count()[["prop_id"]]

In [ ]:
tmp

In [ ]:
len(result04[result04["q7"]==0])

In [ ]:
tempList = []
for i in range(0,len(result04)):
    howMuch = i * 10000
    if result04.loc[i,"q7"]==0:
        tempList.append(0)
    else:
        amountGrade = ((result04.loc[i,"q7"]/10000).astype(int)) * 10000
        tempList.append(amountGrade)
#     else:
#         tempList.append(result04.loc[i,"q7"])

In [ ]:
colName = "q7_grade"
tempDF = pd.DataFrame(tempList, columns=[colName])
result04 = pd.concat([result04, tempDF], axis=1)

In [ ]:
max(result04["q2_c_1"])

In [ ]:
# 금액대 별로 몇 명씩 있는지 볼까?
tmp = result04.groupby(["q7_grade"]).count()[["prop_id"]]

### Decision Tree

In [ ]:
job0 = result04[result04["job_group"]==0]   #대학생
job1 = result04[result04["job_group"]==1]   #직장인
job2 = result04[result04["job_group"]==2]   #무직
job3 = result04[result04["job_group"]==3]   #군인

In [ ]:
len(job3)

In [ ]:
from sklearn import datasets, tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
features = []
#features.append("q3") #여행목적
#features.append("q4_a") #정보획득지
# features.append("q1") # 당일/숙박 여행 여부
# features.append("q8") #사전예약여부
features.append("q7_grade") # 금액대
features

In [ ]:
label = ['q10_grade']
label

In [ ]:
# 데이터 분리 & null값 0으로 대체

# features = 특성, label = 답지
# random 614개(학습할 특성)
trainingData_features = job3[job3.index%4 != 0][features]
trainingData_features = trainingData_features.fillna(0)
# random 614개(학습할 답지)
trainingData_features = job3[job3.index%4 != 0][label]
trainingData_label = trainingData_label.fillna(0)

# random 209개(검증할 특성)
testData_features = job3[job3.index%4 == 0][features]
testData_features = testData_features.fillna(0)
# random 209개(검증할 답지)
# 맨뒤에 [label]이 없는 이유는 전체를 같이 보기 위해서
testData_label = job3[job3.index%4 == 0]
testData_label = testData_label.fillna(0)

In [ ]:
testData = testData_label.reset_index(drop = True)

In [ ]:
# Define Model (모델 정의)
model_method = tree.DecisionTreeRegressor()

# Learning (Feature & Label)
# trainingData_features, trainingData_label을 인자로 던져주면 모델을 만들어줌.
# fit<< 학습시킬때 쓰는 함수, 학습할 특성과 학습할 답지 모두 인자로 던져주어야함.
model = model_method.fit(trainingData_features, trainingData_label)
model

In [ ]:
predict = model.predict(testData_features)
predict

### 데이터프레임으로 변환(DecisionTreeRegression)
predictData = pd.DataFrame(predict)

#컬럼헤더명 바꾸기
predictData.columns = ["DecisionTree Regression"]

#컬럼 인덱스를 통해서 컬럼헤더명 바꾸기
#predictData = predictData.rename(columns = {0:"PREDICT"})
predictData = predictData.reset_index(drop = True)

In [ ]:
resultData = pd.concat([testData, predictData], axis=1)

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(model, out_file=dot_data, 
                rounded=True,
                filled = True,
                feature_names=features,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
sum(job0["q7_grade"])/len(job0["q7_grade"])

In [ ]:
sum(job1["q7_grade"])/len(job1["q7_grade"])

In [ ]:
sum(job2["q7_grade"])/len(job2["q7_grade"])

In [ ]:
sum(job3["q7_grade"])/len(job3["q7_grade"])

In [ ]:
def decisionTreeRegressor(dataName, features, label):
    from sklearn import datasets, tree
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    from sklearn.linear_model import LinearRegression
    
    trainingData_features = dataName[dataName.index%4 != 0][features]
    trainingData_label = dataName[dataName.index%4 != 0][label]
    testData_features = dataName[dataName.index%4 == 0][features]
    testData_label = dataName[dataName.index%4 == 0]

    model_method = tree.DecisionTreeRegressor()
    model = model_method.fit(trainingData_features, trainingData_label)
    decisionTreeRegressor = model.predict(testData_features)

    decisionTreeRegressor = pd.DataFrame(decisionTreeRegressor)
    decisionTreeRegressor.columns = ["PREDICT"]
    decisionTreeRegressorResult = pd.concat([testData_label.reset_index(drop=True), decisionTreeRegressor], axis=1)

    print("MAE : {:.2f}".format(mean_absolute_error(testData_label.reset_index()[label], decisionTreeRegressorResult["PREDICT"])))
    print("MSE : {:.2f}".format(mean_squared_error(testData_label.reset_index()[label], decisionTreeRegressorResult["PREDICT"])))
    print("RMSE : {:.2f}".format(math.sqrt(mean_squared_error(testData_label.reset_index()[label], decisionTreeRegressorResult["PREDICT"]))))
    print("R2 : {:.2f}".format(r2_score(testData_label[label], decisionTreeRegressorResult['PREDICT'])))

    dot_data = StringIO()
    export_graphviz(model, out_file=dot_data, 
                    rounded=True,
                    feature_names = features,
                    filled=True,
                    impurity=False,
                    node_ids=True,
                    proportion=True,
                    special_characters=False)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    return Image(graph.create_png())

In [ ]:
features = []
# features.append("q1") # 당일/숙박 여행 여부
#features.append("q3") #여행목적
#features.append("q4_a") #정보획득지
# features.append("q7_grade") # 금액대
# features.append("q8") #사전예약여부

features

In [ ]:
label = ['q10_grade']
label

In [ ]:
decisionTreeRegressor(job3,features,label)

In [ ]:
yearweekStd = 201630

In [ ]:
trainingData_features = featuresData2[featuresData2.YEARWEEK <= yearweekStd][features]
trainingData_label = featuresData2[featuresData2.YEARWEEK <= yearweekStd][label]
testData_features = featuresData2[featuresData2.YEARWEEK > yearweekStd][features]
testData_label = featuresData2[featuresData2.YEARWEEK > yearweekStd] 

In [ ]:
testData_label = testData_label.reset_index(drop=True)
testData_label

In [ ]:
trainingData_label

In [ ]:
# Define Model
model_method = tree.DecisionTreeRegressor()

# Learning (학습지표 & 결과값)
model = model_method.fit(trainingData_features,trainingData_label)

In [ ]:
dot_data = StringIO()
export_graphviz(model, out_file=dot_data, 
                rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())